# Value Extraction from Financial Statement Documents

Here I have used Regular Expressions to extract the data from the text files 

In [5]:
#importing libraries
import os
import re
import xlsxwriter
import pandas as pd

#taking the file name in the order its been placed in expected format of submission file(i.e. Result.csv)
df=pd.read_csv(r'C:\Users\avinashexpc\Desktop\NLP\final\HCL ML Challenge\Results.csv')

#Writing on excel file which will be submission file in future  
outWorkbook=xlsxwriter.Workbook("C:\\Users\\avinashexpc\\Desktop\\NLP\\final\\HCL ML Challenge\\mu_res.xlsx")
outsheet=outWorkbook.add_worksheet()
outsheet.write(0,0,"Filename")
outsheet.write(0,1,"Extracted Values")
p=0
multi='C:\\Users\\avinashexpc\\Desktop\\NLP\\final\\HCL ML Challenge\\HCL ML Challenge Dataset\\'
for fil in df["Filename"]:
    
    #finding from where to start data extraction from each file
    with open(multi+fil+'.txt') as f:
        lines=f.readlines()
        pattern=re.compile(r'(Notes\s+)|(2020\s\s+)|(2019\s\s+)|(2018\s\s+)|(2017\s\s+)+.*')
        k=0
        t=0
        for line in lines:
            line=line.strip()
            matches=pattern.findall(line)
            #print(matches)
            if line=='Notes' or line=='2020'or line=='2018' or line=='2019'or line=='2017':
                t=1
                break
            if matches!=[]:
                break
            #if(line==)
            k+=1
        #if fil=='X8XZIO5G':
            #break
        #if pattern match is not found then assume data is not appropriate for this pattern 
        #go to next file
        if matches==[] and t==0: #or fil!='X8Y2F0L6':
            p+=1
            print(fil)
            print("^^^^^^^^^^0^^^^^^^^")
            outsheet.write(p,0,fil)
            continue
            
            
        #print(line)
        a=line.find('2018')
        b=line.find('2019')
        c=line.find('2020')
        e=line.find('2017')
        print(fil)
        #print(a," ",b," ",c)
        
        
        #case 1: when balance sheet only have 2019 column
        if a==-1 and c==-1 and e==-1:
            
            
            #creating a pattern for each row of the columns
            pattern=re.compile(r'(([0-9a-zA-ZÂ$€&£(),.:|]+\s)+)\s+([0-9(){},.-]+)\s+')
            lst=[]
            ans={}
            with open(multi+fil+'.txt') as f:
                               lines=f.readlines()
                               i=0
                               for line in lines:
                                    if line=='\n':
                                        i=i+1
                                        continue 
                                    matches=pattern.findall(line)
                                    if matches!=[]:
                                        if matches[0][0]!='Notes ':
                                            it0=matches[0][0]
                                            it1=matches[0][2]
                                            it1=it1.replace("-","nan")
                                            if it1.find("{")==-1 and it1.find("}")==-1:
                                                it1=it1.replace(")","")
                                                it1=it1.replace(",","")
                                                it1=it1.replace("(","-")
                                            it0=it0.replace("$","&#36")
                                            it0=it0.replace("€","&#8364")
                                            it0=it0.replace("£","&#163")
                                            it0=it0.replace("Â","")
                                            matches=[(it0,matches[0][1],it1)]
                                            #print(matches)
                                            ans[i]=matches
                                            lst.append(i)
                                    i=i+1
                                    
            #if pattern match is not found then assume data is not appropriate for this pattern 
            #go to next file                        
            if ans=={}:
               # print("WRONG")
                p+=1
                print(fil)
                print("^^^^^^^^^^0^^^^^^^^")
                outsheet.write(p,0,fil)
                continue
            
            #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
            a=min(k+2,min(lst))
            b=max(lst)
            
            #storing extracted data into dictionary(i.e. res)
            ltr="{"
            res={}
            abc=ans.keys()
            for i in range(a,b+1):
                if lines[i]=='\n':
                    continue 
                if i in ans:
                    xxx=ans[i][0][0]
                    if xxx[0]>='a' and xxx[0]<='z':
                        xxx=it0+xxx
                        if it0 in res:
                            res.pop(it0)
                    res[xxx]=ans[i][0][2]
                else:
                    abb=re.findall(r'\s\s([0-9().,{}-]+)\s\s+',lines[i])
                    #print(abb)
                    if abb!=[]:
                        itt=abb[0]
                        if itt=='-':
                            continue
                        if itt.find("{")==-1 and itt.find("}")==-1:
                            itt=itt.replace(")","")
                            itt=itt.replace(",","")
                            itt=itt.replace("(","-")
                        ltr+='"'+'nan'+'": '+'"'+itt+'", '
                        continue
                    abc=re.findall(r'(([0-9a-zA-ZÂ&$€£:().,-|]+\s)+)',lines[i])
                    it0=abc[0][0]
                    it0=it0.replace("Â","")
                    it0=it0.replace("$","&#36")
                    it0=it0.replace("€","&#8364")
                    it0=it0.replace("£","&#163")
                    res[it0]='nan'
            
            #converting dictionary(i.e res) into string (i.e. ltr)
            for i in res:
                ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
            ltr=ltr[:-2]
            ltr+="}"
            p+=1
            outsheet.write(p,0,fil)
            print(ltr)
            outsheet.write(p,1,ltr)
            print("^^^^^^^^^^1^^^^^^^^")
            continue
            
        #case 2: when balance sheet not have 2019 column
        if b==-1:

            #creating a pattern for each row of the columns
            pattern=re.compile(r'(([0-9a-zA-ZÂ$€£&():,.|]+\s)+)\s+([0-9(){},.-]+)\s+')
            lst=[]
            ans={}
            with open(multi+fil+'.txt') as f:
                               lines=f.readlines()
                               i=0
                               for line in lines:
                                    if line=='\n':
                                        i=i+1
                                        continue 
                                    matches=pattern.findall(line)
                                    if matches!=[]:
                                        if matches[0][0]!='Notes ':
                                            it0=matches[0][0]
                                            it0=it0.replace("$","&#36")
                                            it0=it0.replace("€","&#8364")
                                            it0=it0.replace("£","&#163")
                                            it0=it0.replace("Â","")
                                            matches=[(it0,matches[0][1],"nan")]
                                            #print(matches)
                                            ans[i]=matches
                                            lst.append(i)
                                    i=i+1
                                    
            #if pattern match is not found then assume data is not appropriate for this pattern 
            #go to next file                        
            if ans=={}:
               # print("WRONG")
                p+=1
                print(fil)
                print("^^^^^^^^^^0^^^^^^^^")
                outsheet.write(p,0,fil)
                continue
            
            #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
            a=min(k+2,min(lst))
            b=max(lst)
            
            #storing extracted data into dictionary(i.e. res)
            res={}
            abc=ans.keys()
            for i in range(a,b+1):
                if lines[i]=='\n':
                    continue 
                if i in ans:
                    xxx=ans[i][0][0]
                    if xxx[0]>='a' and xxx[0]<='z':
                        xxx=it0+xxx
                        if it0 in res: 
                            res.pop(it0)
                    res[xxx]=ans[i][0][2]
                else:
                    abc=re.findall(r'(([0-9a-zA-ZÂ&$€£:(),.-|]+\s)+)',lines[i])
                    it0=abc[0][0]
                    it0=it0.replace("Â","")
                    it0=it0.replace("$","&#36")
                    it0=it0.replace("€","&#8364")
                    it0=it0.replace("£","&#163")
                    res[it0]='nan'
                    
            
            #converting dictionary(i.e res) into string (i.e. ltr)
            ltr="{"
            for i in res:
                ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
            ltr=ltr[:-2]
            ltr+="}"
            p+=1
            outsheet.write(p,0,fil)
            print(ltr)
            outsheet.write(p,1,ltr)
            print("^^^^^^^^^^2^^^^^^^^")
            continue
        
        #case 3: when balance sheet has two column among which one is 2019 column
        
        #finding which column is 2019 column
        #if d=1 means 1st column is 2019 else if d=2 2nd column is 2019 column 
        d=0
        if a==-1 and e==-1:
            if b>c:
                d=2
            else:
                d=1
        if c==-1 and e==-1:
            if b>a:
                d=2
            else:
                d=1 
        if a==-1 and c==-1:
            if b>e:
                d=2
            else:
                d=1 
                
        #creating a pattern for each row of the columns
        pattern=re.compile(r'(([0-9a-zA-ZÂ$€&£(),.:|]+\s)+)\s+([0-9(){},.-]+)\s+([0-9(){},.-]+)')
        lst=[]
        ans={}
        with open(multi+fil+'.txt') as f:
                           lines=f.readlines()
                           i=0
                           for line in lines:
                                if line=='\n':
                                    i=i+1
                                    continue 
                                matches=pattern.findall(line)
                                if matches!=[]:
                                    if matches[0][0]!='Notes ':
                                        it0=matches[0][0]
                                        it1=matches[0][2]
                                        it2=matches[0][3]
                                        it2=it2.replace("-","nan")
                                        it1=it1.replace("-","nan")
                                        if it1.find("{")==-1 and it1.find("}")==-1:
                                            it1=it1.replace(")","")
                                            it1=it1.replace(",","")
                                            it1=it1.replace("(","-")
                                        if it2.find("{")==-1 and it2.find("}")==-1:
                                            it2=it2.replace(")","")
                                            it2=it2.replace(",","")
                                            it2=it2.replace("(","-")
                                        it0=it0.replace("Â","")
                                        it0=it0.replace("$","&#36")
                                        it0=it0.replace("€","&#8364")
                                        it0=it0.replace("£","&#163")
                                        matches=[(it0,matches[0][1],it1,it2)]
                                        #print(matches)
                                        ans[i]=matches
                                        lst.append(i)
                                i=i+1
        #if pattern match is not found then assume data is not appropriate for this pattern 
        #go to next file 
        if ans=={}:
           # print("WRONG")
            p+=1
            print(fil)
            print("^^^^^^^^^^0^^^^^^^^")
            outsheet.write(p,0,fil)
            continue
        #print(  )
        #print(ans)
        
        #finding the range of lines or rows need to be extracted (i.e. range(a,b+1))
        a=min(k+2,min(lst))
        b=max(lst)
        print(a,b+1)
        
        #storing extracted data into dictionary(i.e. res)
        ltr="{"
        res={}
        abc=ans.keys()
        for i in range(a,b+1):
            if lines[i]=='\n':
                continue 
            if i in ans:
                #print(i,'*')
                xxx=ans[i][0][0]
                if xxx[0]>='a' and xxx[0]<='z':
                    xxx=it0+xxx
                    if it0 in res:
                        res.pop(it0)
                #print("&&&&",len(ans[i][0]))
                res[xxx]=ans[i][0][d+1]
            else:
                if lines[i].strip()=='year':
                    continue
                abb=re.findall(r'\s\s([0-9(),.{}-]+)\s+([0-9(),.{}-]+\s\s)',lines[i])
                if abb!=[]:
                    #print("hhdbeheb",i)
                    if lines[i+1].strip()=='year':
                        #print("hhdbeheb")
                        iii=it0+'year'
                        if it0 in res:
                            res.pop(it0)
                        itt=abb[0][d-1]
                        itt=itt.replace("-","nan")
                        if itt.find("{")==-1 and itt.find("}")==-1:
                            itt=itt.replace(")","")
                            itt=itt.replace(",","")
                            itt=itt.replace("(","-")
                        ltr+='"'+iii+'": '+'"'+itt+'", '
                        continue
                    itt=abb[0][d-1]
                    if itt=='-':
                        continue
                    if itt.find("{")==-1 and itt.find("}")==-1:
                        itt=itt.replace(")","")
                        itt=itt.replace(",","")
                        itt=itt.replace("(","-")
                    ltr+='"'+'nan'+'": '+'"'+itt+'", '
                    continue
                abc=re.findall(r'(([0-9a-zA-ZÂ&$€£:,().-|]+\s)+)',lines[i])
                it0=abc[0][0]
                it0=it0.replace("Â","")
                it0=it0.replace("$","&#36")
                it0=it0.replace("€","&#8364")
                it0=it0.replace("£","&#163")
                #print("****",it0,i)
                res[it0]='nan'
        
        
        #converting dictionary(i.e res) into string (i.e. ltr)
        for i in res:
            ltr+='"'+str(i)[:-1]+'": '+'"'+str(res[i])+'", '
        ltr=ltr[:-2]
        ltr+="}"
        p+=1
        outsheet.write(p,0,fil)
        print(ltr)
        outsheet.write(p,1,ltr)
        print("^^^^^^^^^3^^^^^^^^^")
outWorkbook.close()



X8XUG1JD
3 9
{"Current Assets": "312", "Creditors: amounts falling due within one year": "-59", "Net current assets (liabilities)": "253", "Total assets less current liabilities": "253", "Total net assets (liabilities)": "253", "Capital and reserves": "253"}
^^^^^^^^^3^^^^^^^^^
X8XX0C57
3 8
{"Called up share capital not paid": "1", "Net assets": "1", "Tssued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8XX02AX
3 7
{"Net assets": "0", "Issued share capital": "nan", "1 Ordinary Share 0 of &#163 each": "0", "Total Shareholder funds": "0"}
^^^^^^^^^3^^^^^^^^^
X8XX0ANN
3 8
{"Called up share capital not paid": "1", "Net assets": "1", "Tssued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8XX0BU1
3 9
{"Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Issued share capital": "nan", "100 Ordinary Shares 1 of &#163 each": "10

X8XZIBXE
3 5
{"Net assets": "0", "Reserves": "0"}
^^^^^^^^^3^^^^^^^^^
X8XZIBZD
3 9
{"Called up share capital not paid": "1", "Current Assets": "1016", "Net current assets (liabilities)": "1016", "Total assets less current liabilities": "1017", "Total net assets (liabilities)": "1017", "Capital and reserves": "1017"}
^^^^^^^^^3^^^^^^^^^
X8XZICIB
3 9
{"Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Issued share capital": "nan", "100 Ordinary Shares of &#163 | each": "100", "Total Shareholder funds": "100"}
^^^^^^^^^3^^^^^^^^^
X8XZICNE
3 9
{"Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Issued share capital": "nan", "100 Ordinary Shares 1 of &#163 each": "100", "Total Shareholder funds": "100"}
^^^^^^^^^3^^^^^^^^^
X8XZICQX
3 9
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^

{"Called up share capital not paid": "1", "Net assets": "1", "Tssued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^^1^^^^^^^^
X8XZIP1V
4 15
{"Current assets": "nan", "Cash at bank and in hand 2": "nan", "Creditors: amounts falling due within one year": "5", "Net current assets": "nan", "Net assets": "nan", "Capital and reserves": "nan", "Called up share capital": "3", "Shareholders funds": "nan"}
^^^^^^^^^3^^^^^^^^^
X8XZIPBM
3 13
{"Creditors: amounts falling due after more than one year": "-37.070", "Fixed Assets": "83750", "Current Assets": "93152", "Creditors: amounts falling due within one year": "-29819", "Net current assets (liabilities)": "63333", "Total assets less current liabilities": "147083", "Total net assets (liabilities)": "110013", "Capital and reserves": "110013"}
^^^^^^^^^3^^^^^^^^^
X8XZIPGP
3 7
{"Called up share capital not paid": "100", "Net assets": "100", "Tssued share capital": "nan", "100 Shares 1 of &#16

{"Called up share capital not paid": "0", "Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Authorised share capital": "nan", "100 ORDINARY SHARES of &#1631 each": "100", "Issued share capital": "nan", "Total shareholder funds": "100"}
^^^^^^^^^^1^^^^^^^^
X8Y2EYGW
3 9
{"Current Assets": "28422", "Creditors: amounts falling due within one year": "-28420", "Net current assets (liabilities)": "2", "Total assets less current liabilities": "2", "Total net assets (liabilities)": "2", "Capital and reserves": "2"}
^^^^^^^^^3^^^^^^^^^
X8Y2EYNK
{"Called up share capital not paid": "0", "Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Authorised share capital": "nan", "100 ORDINARY SHARES of &#1631 each": "100", "Issued share capital": "nan", "Total shareholder funds": "100"}
^^^^^^^^^^1^^^^^^^^
X8Y2EZ9S
3 8
{"Called up share capital not paid": "1", "Net assets": "1", "Tssued share capital": "nan", "1 Ordinary Share 1 of &#163 each"

3 9
{"Current Assets": "677", "Creditors: amounts falling due within one year": "nan", "Net current assets (liabilities)": "366", "Total assets less current liabilities": "366", "Total net assets (liabilities)": "366", "Capital and reserves": "366"}
^^^^^^^^^3^^^^^^^^^
X8Y4QT61
4 14
{"Assets": "nan", "Unpaid called up share capital, classed as current asset": "1.00", "Cash at bank and in hand": "0.00", "Net assets": "1.00", "Capital & reserves": "nan", "Fully paid called up share capital": "0.00", "Shareholder funds": "1.00"}
^^^^^^^^^3^^^^^^^^^
X8Y4QUNU
3 10
{"Called up share capital not paid": "1", "Fixed Assets": "97", "Current Assets": "nan", "Net current assets (liabilities)": "0", "Total assets less current liabilities": "98", "Total net assets (liabilities)": "98", "Capital and reserves": "98"}
^^^^^^^^^3^^^^^^^^^
X8Y4QUTD
{"Called up share capital not paid": "3", "Net assets": "3", "Tssued share capital": "nan", "3 Ordinary Shares 1 of &#163 each": "3", "Total Shareholder funds

...

3 9
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8Y4ULYO
3 5
{"Net assets": "0", "Reserves": "0"}
^^^^^^^^^3^^^^^^^^^
X8Y4UMQ0
3 9
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8Y4UN56
4 19
{"nan": "59683", "Fixed assets": "23938", "Current assets": "58525", "Prepayments and accrued income": "1158", "Creditors: amounts falling due within one year": "-62754", "Creditors: amounts falling due within assets": "-3071", "Total assets less current liabilities": "20867", "Creditors: amounts falling due after more than one year": "-12703", "Accruals and deferred income": "-3625", "Net assets": "4539", "Capital and reserves": "4539", "Number": "nan", "Average number of employee": "5"}
^

X8Y4V99N
{"Called up share capital not paid": "0", "Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Authorised share capital": "nan", "100 ORDINARY SHARES of &#1631 each": "100", "Issued share capital": "nan", "Total shareholder funds": "100"}
^^^^^^^^^^1^^^^^^^^
X8Y4VBTE
3 9
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8Y4VCSX
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^^1^^^^^^^^
X8Y4VEBL
3 5
{"Net assets": "0", "Reserves": "0"}
^^^^^^^^^3^^^^^^^^^
X8Y4VEGH
3 8
{"Called up share capital not paid": "1", "Net assets": "1", "Tssued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8Y4VEGP


3 8
{"Called up share capital not paid": "100", "Net assets": "100", "Tssued share capital": "nan", "100 Ordinary Shares of &#163 | each": "100", "Total Shareholder funds": "100"}
^^^^^^^^^3^^^^^^^^^
X8Y4W3ZF
{"Fixed assets": "2378", "Current assets": "28355", "Creditors: amounts falling due within one year": "-20194", "Net current assets": "8161", "Total assets less current liabilities": "10539", "Accruals and deferred income": "-2740", "Net assets": "7799", "Capital and reserves": "7799"}
^^^^^^^^^^1^^^^^^^^
X8Y4W423
3 11
{"Fixed Assets": "290", "Current Assets": "2090", "Creditors: amounts falling due within one year": "-568", "Net current assets (liabilities)": "1522", "Total assets less current liabilities": "1812", "Accruals and deferred income": "-780", "Total net assets (liabilities)": "1032", "Reserves": "1032"}
^^^^^^^^^3^^^^^^^^^
X8Y4W4EP
3 12
{"Creditors: amounts falling due after more than one year": "-57.560", "Fixed Assets": "38260", "Current Assets": "6200", "Net curren

{"Current Assets": "8278", "Prepayments and accrued income": "1", "Creditors: amounts falling due within one year": "-8212", "Net current assets (liabilities)": "67", "Total assets less current liabilities": "67", "Total net assets (liabilities)": "67", "Capital and reserves": "67"}
^^^^^^^^^^1^^^^^^^^
X8Y4SXZU
3 9
{"Current assets": "nan", "Cash at bank and in hand": "2", "Net assets": "2", "Issued share capital": "nan", "2 Ordinary Shares 1 of &#163 each": "2", "Total Shareholder funds": "2"}
^^^^^^^^^3^^^^^^^^^
X8Y4SYH4
{"Current assets": "nan", "Cash at bank and in hand": "100", "Net assets": "100", "Issued share capital": "nan", "100 Ordinary Shares 1 of &#163 each": "100", "Total Shareholder funds": "100"}
^^^^^^^^^^1^^^^^^^^
X8Y4SZ5L
3 9
{"Current assets": "nan", "Cash at bank and in hand": "1", "Net assets": "1", "Issued share capital": "nan", "1 Ordinary Share 1 of &#163 each": "1", "Total Shareholder funds": "1"}
^^^^^^^^^3^^^^^^^^^
X8Y4SZOY
3 8
{"Called up share capital not 

3 8
{"Called up share capital not paid": "100", "Net assets": "100", "Tssued share capital": "nan", "100 Ordinary Shares of &#163 | each": "100", "Total Shareholder funds": "100"}
^^^^^^^^^3^^^^^^^^^
X8Y4TKPN
3 12
{"Creditors: amounts falling due after more than one year": "-90.628", "Current Assets": "2124", "Creditors: amounts falling due within one year": "-250", "Net current assets (liabilities)": "1874", "Total assets less current liabilities": "1874", "Total net assets (liabilities)": "-88754", "Capital and reserves": "-88754"}
^^^^^^^^^3^^^^^^^^^
X8Y4TLIR
3 10
{"Called up share capital not paid": "1", "Current Assets": "320", "Prepayments and accrued income": "135", "Net current assets (liabilities)": "455", "Total assets less current liabilities": "456", "Total net assets (liabilities)": "456", "Capital and reserves": "456"}
^^^^^^^^^3^^^^^^^^^
X8Y4TOYI
3 10
{"Fixed Assets": "53", "Current Assets": "3405", "Creditors: amounts falling due within one year": "-360", "Net current a

## Observations

1) My code is not being able to extract information from all the txt files.(i.e. datas from few files were not extracted)                  
2) Many of my extracted datas are not being able to convert into scores. 

## Conclusion 

This is my first Machine Learning Challenge in which i have participated, though my score is not good but i went too close to get my result correct which is good for me. I had learnt a lot of new stuffs like regular expression, jupyter notebook, json and lot more.    

## About Myself

I am B.tech 3rd year student of NIT Durgapur. I am a competitive coder working on c/c++/python. In recent time i had worked on few ML projects in summer internship at IIT Kharagpur, where i worked on unsupervised learning(i.e. ICA,PCA,SVD..and lot more). After that i found intrest in this field and trying to explore... I would be honoured to get any project under your guidance in summer 2020, to gain experience and knowledge through the project. Hoping for your kind consideration.
